#### fire pulse curve fit
Loads fire pulse data and does some curve fitting and other analysis.

Follow the curve fit example from [pycse](http://kitchingroup.cheme.cmu.edu/pycse/pycse.html), paragraph 7.5

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import leastsq
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

## Load fire pulse data from files
Fire pulse data was previously sliced from much longer data sets and saved to files.  Here only the main pulse is loaded.

In [2]:
df1 = pd.read_pickle('FIRE_800V.pkl')
df2 = pd.read_pickle('FIRE_720V.pkl')
df3 = pd.read_pickle('FTSA Fire Pulse.pkl')

## Normalize and time align the data
- normalize data to 1
- convert time base to nsec
- time align peak to t=0

In [3]:
# FIRE 800V
x1 = df1['Time'].values
y1 = df1['Voltage'].values
y1 = y1/y1.max()
x1 *= 1e9
y1_peak_index = y1.argmax()
x1 -= x1[y1_peak_index]

In [4]:
# FIRE 720V
x2 = df2['Time'].values
y2 = df2['Voltage'].values
y2 = y2/y2.max()
x2 *= 1e9
y2_peak_index = y2.argmax()
x2 -= x2[y2_peak_index]

In [5]:
# CVR Fire Pulse
x3 = df3['Time'].values
y3 = df3['Voltage'].values
y3 = y3/y3.max()
y3_peak_index = y3.argmax()
x3 -= x3[y3_peak_index]

## Plot the data

In [6]:
# plot the fire pulse
plt.plot(x1,y1,'o', markersize=1)
plt.plot(x2,y2,'o', markersize=1)
plt.plot(x3,y3,'o', markersize=1)
plt.xlabel('Time, nsec')
plt.ylabel('Fire pulse, normalized')
plt.legend(['FIRE_800V', 'FIRE_720V', 'CVR Fire Pulse'], loc='best')
plt.title('Fire Pulse, normalized and time aligned to peak')
plt.show()

## Plotting only the leading edge of the fire pulses

In [7]:
# plot the fire pulse
plt.plot(x1[y1_peak_index-1500:y1_peak_index],y1[y1_peak_index-1500:y1_peak_index],'o', markersize=1)
plt.plot(x2[y2_peak_index-300:y2_peak_index],y2[y2_peak_index-300:y2_peak_index],'o', markersize=1)
plt.plot(x3[y3_peak_index-120:y3_peak_index],y3[y3_peak_index-120:y3_peak_index],'o', markersize=1)
plt.xlabel('Time, nsec')
plt.ylabel('Fire pulse, normalized')
plt.legend(['FIRE_800V', 'FIRE_720V', 'CVR Fire Pulse'], loc='best')
plt.title('Fire Pulse leading edge, normalized and time aligned to peak')
plt.show()

## Plot the main body of the pulse

In [8]:
# plot the fire pulse
start = 800
end = 900
x1_main = x1[y1_peak_index-start:y1_peak_index+end]
y1_main = y1[y1_peak_index-start:y1_peak_index+end]

plt.plot(x1_main,y1_main,'o', markersize=1)
#plt.plot(x2[y2_peak_index-300:y2_peak_index],y2[y2_peak_index-300:y2_peak_index],'o', markersize=1)
#plt.plot(x3[y3_peak_index-120:y3_peak_index],y3[y3_peak_index-120:y3_peak_index],'o', markersize=1)
plt.xlabel('Time, nsec')
plt.ylabel('Fire pulse, normalized')
plt.legend(['FIRE_800V'], loc='best')
plt.title('Fire Pulse')
plt.show()

In [9]:
# adjust the x axis so that all values of time are positive
x1_main -= x1_main[0]

In [10]:
# fitting the FIRE 800V data to a poly
z = np.polyfit(x1_main, y1_main, 2)
z

array([ -1.38814079e-04,   1.78382436e-02,   3.07343708e-01])

In [11]:
# convert it to a function
fx = np.poly1d(z)

In [12]:
#plot the poly
xp = np.linspace(x1_main[0], x1_main[-1], 100)
plt.plot(xp, fx(xp), '-')
plt.plot(x1_main,y1_main,'ro', markersize=1)
plt.xlabel('Time, nsec')
plt.ylabel('fire pulse, volts')
plt.legend(['poly fit', 'data'], loc='best')
plt.show()

### plot the poly over a longer time frame

In [19]:
#plot the poly
xp = np.linspace(0, 200, 500)
plt.plot(xp, fx(xp), '-')
plt.plot(x1_main,y1_main,'ro', markersize=1)
plt.xlabel('Time, nsec')
plt.ylabel('fire pulse, volts')
plt.legend(['poly fit', 'data'], loc='best')
plt.show()

### converting poly coeff to circuit elements

In [76]:
# following the example of paragraph 7.5
t1 = np.linspace(x1_main[0], x1_main[-1], 100)
c1 = fx(t1)

def rlc(parameters, t1):
    'From Phys. Rev. B 28, 5480 (1983)'
    R, C, L = parameters
    alpha = R/(2*L)
    wd = np.sqrt(1/(L*C) - alpha**2)
    I = np.exp(-alpha*t1)*np.sin(wd*t1)
    return I

def objective(pars, y, x):
    #we will minimize this function
    err =  y - rlc(pars, x)
    return err

#x0 = [0.15, 0.12e-6, 1e-9] #initial guess of parameters, R, C & L
x0 = [.5, 1e-6, 1e-9] #initial guess of parameters, R, C & L

plsq = leastsq(objective, x0, args=(c1, t1))

print('Fitted parameters = {0}'.format(plsq[0]))


Fitted parameters = [  5.00000000e-01   1.00000000e-06   1.00000000e-09]


/ext/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


In [77]:
print(fx)

            2
-0.0001717 x + 0.02122 x + 0.253


In [78]:
print(z)

[ -1.71671544e-04   2.12176239e-02   2.53019200e-01]


In [79]:
#fx.roots()
np.poly1d(z).roots

array([ 134.54839588,  -10.95409845])

# OLD CODE BELOW HERE

In [2]:
# define the data frame
time = []
current = []
voltage = []
df = pd.DataFrame(columns=['Time','Current','Voltage'])

In [3]:
# read CSV
csv1 = pd.read_csv('sample ringdown.csv')

In [4]:
time = csv1['Time'].values
time = time.astype(float)

current = csv1['Current'].values
current = current.astype(float)

voltage = csv1['Voltage'].values
voltage = voltage.astype(float)

In [5]:
# put data into the frame
df = pd.DataFrame({
    'Time' : time,
    'Current' : current,
    'Voltage' : voltage
    })

In [6]:
print('number of points in the data set: {:d}'.format(csv1['Time'].count()))

number of points in the data set: 2270


In [7]:
# store the data frame as a pickle file
df.to_pickle('sample ringdown.pkl')

In [7]:
# plot some of the data
num_samples = 1000
#plt.plot(df['Time'].values, df['Voltage'].values)
plt.plot(df['Time'].values[0:num_samples], df['Current'].values[0:num_samples],'ro',markersize=1)
plt.xlabel('Time, sec')
plt.ylabel('Current, amps')
plt.show()

#### The Series RLC Circuit

The loop equation for the circuit is: $L\frac{di}{dt} + Ri + \frac{1}{C}\int_{0}^{i} idt + Vo = 0$

The circuit is under damped: $C \lt \frac{4L}{R^2}$

The resonant frequency is: $\omega _d = \frac{1}{\sqrt{LC}}$

The damping coefficient is: $\alpha = \frac{R}{2L}$

The damped frequency is: $\omega _d = \sqrt{\omega _{0}^{2}-\alpha ^{2}}$

The underdamped responce is: $i = e^{-\alpha t}(A_{1}cos\omega _d t + A_{2}sin\omega _d t)$

Intial capacitor voltage is 1000 volts.  

In [34]:
# following the example of paragraph 7.5
num_samples = 500
t1 = df['Time'].values[0:num_samples]
c1 = df['Current'].values[0:num_samples]

def rlc(parameters, t1):
    'From Phys. Rev. B 28, 5480 (1983)'
    R, L, C = parameters
    alpha = R/(2*L)
    wd = np.sqrt(1/(L*C) - alpha**2)
    I = 2000*np.exp(-alpha*t1)*np.sin(wd*t1)
    return I

def objective(pars, y, x):
    #we will minimize this function
    err =  y - rlc(pars, x)
    return err

x0 = [0.5, 0.22e-6, 10e-9] #initial guess of parameters

plsq = leastsq(objective, x0, args=(c1, t1))

print('Fitted parameters = {0}'.format(plsq[0]))


Fitted parameters = [  4.18780866e-01   2.06667652e-07   1.07039042e-08]


/ext/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in sqrt


In [35]:
num_samples = 500
t1 = df['Time'].values[0:num_samples]
y = rlc(plsq[0], t1)
plt.plot(t1, y)
plt.plot(df['Time'].values[0:num_samples], df['Current'].values[0:num_samples],'ro',markersize=1)